In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tarfile

import shutil
import yaml

from functools import partial
from glob import glob, iglob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections.shapefiles import load_shapes, iter_records
from projections.models import Records
from projections.elevation import get_indices_by_file
from projections import raster, utils


pd.set_option('max_columns', None)

/sd2/Moncho/zonalstats_earth_engine/.venv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
def save_location_mapping(row_and_path):
    row, path = row_and_path
    shape = row['geometry']
    
    IMAGE = utils.read_tif(read_path / base_file)
    subset = raster.find_subset_with_intersection_area(IMAGE, shape)

    if subset.empty:
        with open(path, 'w') as f:
            f.write('')
        return

    subset['id'] = row['id']
    
    subset.to_csv(path, index=False)        

In [3]:
base_file = "Alfalfa.tif"
read_path = Path('../Data/FAO/Agro-ClimaticPotentialYield')
output_path = Path('../Output/FAO/Agro-ClimaticPotentialYield/')
partial_path = output_path / 'partial'
by_country_path = output_path / 'by_country'

output_path.mkdir(exist_ok=True)
partial_path.mkdir(exist_ok=True)
by_country_path.mkdir(exist_ok=True)

In [4]:
# Validate that all TIF share the same coordinates
x = None
y = None

for tif in read_path.glob('*.tif'):
    image = utils.read_tif(tif)
    if x is None:
        x = image.x
        y = image.y
    else:
        assert np.all(x == image.x) and np.all(y == image.y), tif
        
no_data_value = image._FillValue

# Map raster to polygons

In [5]:
geo_df = gpd.read_file('../Shapefiles/preprocessed/all_countries_with_eth.shp')

In [6]:
n_processes = 30

# IMAGE = utils.read_tif(read_path / base_file)

iterator = partial(utils.yield_missing_shapes, save_path=partial_path, prefix=base_file[:-4])

if n_processes == 1:
    for row_and_path in tqdm(iterator(geo_df)):
        save_location_mapping(row_and_path)
else:
    with ProcessPoolExecutor(n_processes) as tpe:
        for _ in tqdm(
            tpe.map(save_location_mapping, iterator(geo_df)), 
            total=geo_df.shape[0]
        ):
            pass

100%|███████████████████████████████████| 122772/122772 [40:37<00:00, 50.37it/s]


# Union portions from different files and shapes

In [7]:
df_by_region = {}
for file in tqdm(partial_path.glob('*.csv'), desc='Reading'):
    try:
        df = pd.read_csv(file)
    except pd.errors.EmptyDataError:
        continue
        
    if 'id' not in df.columns:
        df['id'] = df['adm2']
        df['id'].fillna(df['adm1'], inplace=True)
        df['id'].fillna(df['adm0'], inplace=True)
    region = df.loc[0, 'id']
    df_by_region.setdefault(region, []).append(df)

for region, dfs in tqdm(df_by_region.items(), desc='Saving'):
    df = utils.combine_dataframes(dfs)
    df.to_feather(by_country_path / f'{region}.feather')
    
del df_by_region

Reading: 122772it [03:28, 588.59it/s]
Saving: 100%|████████████████████████████| 48140/48140 [01:14<00:00, 645.83it/s]


# Aggregate

In [8]:
utils.aggregate_feather_splits_and_save(by_country_path, output_path / base_file, no_data_value=-9999)

Grouping: 100%|██████████████████████████| 48140/48140 [03:34<00:00, 223.95it/s]


# Additional files with same raster

In [9]:
for file in read_path.glob('*.tif'):
    print(file.name)
    file_path = output_path / file.name[:-4]
    file_path.mkdir(exist_ok=True)

    IMAGE = utils.read_tif(file)
    increment = raster.get_increment_from_tif(IMAGE)
    
    for df_path in tqdm(by_country_path.glob('*.feather')):
        df = pd.read_feather(df_path)
        pol = utils.get_mock_polygon_from_df(df, increment=increment)
        subdf = raster.merge_df_to_array_by_lat_lon(df, IMAGE, pol)
        if subdf.empty:
            print(df_path.name, 'is empty')
        else:
            subdf.to_feather(file_path / df_path.name)
            
    utils.aggregate_feather_splits_and_save(file_path, output_path / file.name, no_data_value=-9999)
    shutil.rmtree(file_path)

Alfalfa.tif


48140it [07:42, 104.00it/s]
Grouping: 100%|██████████████████████████| 48140/48140 [03:27<00:00, 231.87it/s]


Banana.tif


48140it [08:14, 97.32it/s] 
Grouping: 100%|██████████████████████████| 48140/48140 [03:28<00:00, 230.58it/s]


Barley.tif


...

48140it [07:50, 102.24it/s]
Grouping: 100%|██████████████████████████| 48140/48140 [03:45<00:00, 213.11it/s]


Yam.tif


48140it [07:51, 102.15it/s]
Grouping: 100%|██████████████████████████| 48140/48140 [03:53<00:00, 206.56it/s]


Yellow_Yam.tif


48140it [07:53, 101.56it/s]
Grouping: 100%|██████████████████████████| 48140/48140 [03:42<00:00, 216.21it/s]
